<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/modeling_MLPMixer/test_sample_MLPMixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

In [2]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [1]:
# ! rm -rf PyTorch-Architectures/

In [ ]:
! kaggle competitions download -c dogs-vs-cats

In [ ]:
# ! kaggle datasets download -d dansbecker/hot-dog-not-hot-dog
# ! unzip -q hot-dog-not-hot-dog.zip

In [3]:
! unzip -q train.zip
! rm train.zip
! rm test1.zip

In [1]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/modeling_MLPMixer

fatal: destination path 'PyTorch-Architectures' already exists and is not an empty directory.
/content/PyTorch-Architectures/modeling_MLPMixer


In [5]:
! mv /content/train .
# ! mv /content/test1 .

In [2]:
import os
import time
from PIL import Image
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from model import MLPMixer

In [3]:
samples = os.listdir('train')
print('Total Samples: ', len(samples))

Total Samples:  25000


In [4]:
parent_path = '/content/PyTorch-Architectures/modeling_MLPMixer/train/'
samples = [parent_path + str(sample) for sample in samples]
samples = [(sample, 1) if 'dog' in sample else (sample, 0) for sample in samples]

In [5]:
random.shuffle(samples)

split = 90 * len(samples) // 100
train_samples = samples[:split]
valid_samples = samples[split:]

print('Train Samples: ', len(train_samples))
print('Valid Samples: ', len(valid_samples))

Train Samples:  22500
Valid Samples:  2500


In [6]:
class CustomDataset(Dataset):
  def __init__(self, path_images, transforms=None):
    self.path_images = path_images
    self.transforms = transforms
  
  def __len__(self):
    return len(self.path_images)
  
  def __getitem__(self, idx):
    images, labels = self.path_images[idx][0], self.path_images[idx][1]
    images = Image.open(images)
    if self.transforms is not None:
      images = self.transforms(images)
    return {
        'img': images,
        'labels': torch.tensor(labels),
    }

transformations = transforms.Compose([
                                 transforms.Resize(size=(256, 256)),
                                 transforms.ToTensor(),
])

In [7]:
sample_dataset = CustomDataset(path_images=train_samples, transforms=transformations)
sample_loader = DataLoader(dataset=sample_dataset, batch_size=2)
for sample in sample_loader:
  print(sample['img'].shape, sample['labels'].shape)
  print(sample['labels'])
  break

torch.Size([2, 3, 256, 256]) torch.Size([2])
tensor([1, 0])


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = MLPMixer(image_size=256,
                 patch_size=16,
                 channel=3,
                 dim=512,
                 depth=8,
                 num_classes=2)
model.to(device)

In [9]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Trainable Parameters: ', params)

Trainable Parameters:  3613698


In [10]:
# Hyperparameter section
EPOCHS = 3
LR = 3e-4
BATCH_SIZE = 64

In [11]:
train_dataset = CustomDataset(path_images=train_samples, transforms=transformations)
valid_dataset = CustomDataset(path_images=valid_samples, transforms=transformations)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

print('Length of Train Loader: ', len(train_loader))
print('Length of Valid Loader: ', len(valid_loader))

Length of Train Loader:  352
Length of Valid Loader:  40


In [12]:
# Sanity check forward pass
model.eval()
with torch.set_grad_enabled(False):
  imgs = sample['img'].to(device)
  labels = sample['labels'].to(device)
  outputs = model(img=imgs)
  assert outputs.size(0) == imgs.size(0), "Batch sizes don't match"

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [16]:
def compute_accuracy(model, data_loader, device):
  correct, total = 0, 0
  with torch.set_grad_enabled(False):
    for sample in data_loader:
      imgs = sample['img'].to(device)
      labels = sample['labels'].to(device)
      logits = model(imgs)
      prob = F.softmax(logits, dim=-1)
      _, preds = torch.max(prob, 1)
      correct += (preds == labels).sum()
      total += labels.size(0)
  return correct.float() / total * 100

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    imgs = sample['img'].to(device)
    labels = sample['labels'].to(device)
    logits = model(imgs)
    loss = F.cross_entropy(logits, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if idx % 100 == 0:
      print('Epoch: %04d/%04d || Batch: %04d/%04d || Loss: %.2f' % (epoch+1,
                                                                    EPOCHS,
                                                                    idx,
                                                                    len(train_loader),
                                                                    loss.item()))
  model.eval()
  train_acc = compute_accuracy(model, train_loader, device)
  valid_acc = compute_accuracy(model, valid_loader, device)
  print('Train Accuracy: %.2f%% || Valid Accuracy: %.2f%%' % (train_acc.item(),
                                                              valid_acc.item()))
  epoch_elapsed_time = (time.time() - start_time) / 60
  print('Epoch Elapsed Time: %.2f min' % (epoch_elapsed_time))
total_training_time = (time.time() - start_time) / 60
print('Total Training Time: %.2f min' % (total_training_time))

Epoch: 0001/0003 || Batch: 0000/0352 || Loss: 0.69
Epoch: 0001/0003 || Batch: 0100/0352 || Loss: 0.70
Epoch: 0001/0003 || Batch: 0200/0352 || Loss: 0.72
Epoch: 0001/0003 || Batch: 0300/0352 || Loss: 0.72
Train Accuracy: 62.27% || Valid Accuracy: 62.20%
Epoch Elapsed Time: 8.39 min
Epoch: 0002/0003 || Batch: 0000/0352 || Loss: 0.69
Epoch: 0002/0003 || Batch: 0100/0352 || Loss: 0.62
Epoch: 0002/0003 || Batch: 0200/0352 || Loss: 0.63
Epoch: 0002/0003 || Batch: 0300/0352 || Loss: 0.68
Train Accuracy: 66.47% || Valid Accuracy: 64.92%
Epoch Elapsed Time: 16.79 min
Epoch: 0003/0003 || Batch: 0000/0352 || Loss: 0.65
Epoch: 0003/0003 || Batch: 0100/0352 || Loss: 0.58
Epoch: 0003/0003 || Batch: 0200/0352 || Loss: 0.59
Epoch: 0003/0003 || Batch: 0300/0352 || Loss: 0.61
Train Accuracy: 72.18% || Valid Accuracy: 69.92%
Epoch Elapsed Time: 25.20 min
Total Training Time: 25.20 min
